# Prepare the PandaRoot Simulation Data

In [4]:
# python IO for ROOT files
import uproot

import os
from concurrent.futures import ProcessPoolExecutor

# numpy
import numpy as np

# For interactive plotting
import matplotlib.pyplot as plt
from matplotlib import patches
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Seaborn for plotting and styling
import seaborn as sns

# Pandas for data frame manipulation
import pandas as pd

from utils.data_processing import (
    get_all_mother_ids,
    get_process_ids,
    is_signal_particle,
    get_process_name,
    get_particle_tex_name,
)
from tqdm import tqdm

In [5]:
simFile = "/home/nikin105/mlProject/data/simulations/XiAntiXi/root/XiAntiXi_sim.root"
print("Input simulation file: ", simFile)

Input simulation file:  /home/nikin105/mlProject/data/simulations/XiAntiXi/root/XiAntiXi_sim.root


In [6]:
sim_tree = uproot.open(
    simFile + ":pndsim",
)

In [7]:
signal_processes = [
    [88888, 3312],  # ppbar -> Xi-
    [88888, -3312],  # ppbar -> anti-Xi+
    [88888, 3312, -211],  # ppbar -> Xi- -> pi-
    [88888, -3312, 211],  # ppbar -> anti-Xi+ -> pi+
    [88888, 3312, 3122],  # ppbar -> Xi- -> Lambda0
    [88888, -3312, -3122],  # ppbar -> anti-Xi+ -> anti-Lambda0
    [88888, 3312, 3122, -211],  # ppbar -> Xi- -> Lambda0 -> pi-
    [88888, -3312, -3122, 211],  # ppbar -> anti-Xi+ -> anti-Lambda0 -> pi+
    [88888, 3312, 3122, 2212],  # ppbar -> Xi- -> Lambda0 -> p
    [88888, -3312, -3122, -2212],  # ppbar -> anti-Xi+ -> anti-Lambda0 -> anti-p
]

# signal_processes = [[13], [-13]]  # muon, anti-muon


# signal_process_ids = [
#     [0, 0],
#     [0, 0, 4],
#     [0, 0, 4, 4],
# ]

signal_process_ids = [
    [0, 0],
    [0, 0, 0],
    [0, 0, 0, 0],
]

# signal_process_ids = [[0]]

In [8]:
tqdm.pandas()

momentum_keys = [
    "MCTrack.fPx",
    "MCTrack.fPy",
    "MCTrack.fPz",
]

pid_keys = [
    "MCTrack.fMotherID",
    "MCTrack.fSecondMotherID",
    "MCTrack.fProcess",
    "MCTrack.fPdgCode",
]
stt_keys = ["STTPoint.fTrackID"]

momenta = sim_tree.arrays(
    expressions=momentum_keys + pid_keys + stt_keys, library="pd"
)

my_dtypes = {
    "MCTrack.fPx": "float32",
    "MCTrack.fPy": "float32",
    "MCTrack.fPz": "float32",
    "MCTrack.fPdgCode": "int32",
}

In [9]:
def compute_row(row) -> pd.Series:
    output_dict = {}
    unique_track_ids = np.unique(np.array(row["STTPoint.fTrackID"], dtype=int))
    for key in momentum_keys + ["MCTrack.fPdgCode"]:
        output_dict[key] = np.array(row[key][unique_track_ids], dtype=my_dtypes[key])

    mother_ids = get_all_mother_ids(
        mother_ids=row["MCTrack.fMotherID"],
        second_mother_ids=row["MCTrack.fSecondMotherID"],
    )

    output_dict["is_signal"] = np.zeros(len(unique_track_ids), dtype=bool)
    output_dict["particle_name"] = []
    output_dict["production_process"] = []

    particle_num = 0
    for particle_id in unique_track_ids:
        mc_ids, process_codes = get_process_ids(
            process_ids=np.array(row["MCTrack.fProcess"], dtype=int),
            mother_ids=mother_ids,
            pdg_ids=np.array(row["MCTrack.fPdgCode"], dtype=int),
            particle_id=particle_id,
        )
        output_dict["is_signal"][particle_num] = is_signal_particle(
            process_mc_ids=mc_ids,
            process_ids=process_codes,
            signal_mc_ids=signal_processes,
            signal_process_ids=signal_process_ids,
        )
        output_dict["production_process"].append(
            get_process_name(process_id=process_codes[-1])
        )
        output_dict["particle_name"].append(
            "$"
            + get_particle_tex_name(pdg_id=np.array(row["MCTrack.fPdgCode"][particle_id], dtype=int))
            + "$"
        )

        output_dict["pt"] = np.sqrt(
            output_dict["MCTrack.fPx"] ** 2 + output_dict["MCTrack.fPy"] ** 2
        )
        output_dict["P"] = np.sqrt(
            output_dict["MCTrack.fPx"] ** 2
            + output_dict["MCTrack.fPy"] ** 2
            + output_dict["MCTrack.fPz"] ** 2
        )
        output_dict["theta"] = np.arctan2(output_dict["pt"], output_dict["MCTrack.fPz"])
        output_dict["phi"] = np.arctan2(
            output_dict["MCTrack.fPy"], output_dict["MCTrack.fPx"]
        )
        output_dict["eta"] = -np.log(np.tan(output_dict["theta"] / 2))

        particle_num += 1

    return pd.Series(output_dict)

In [10]:
def process_dataframe_in_parallel(df):
    # Initialize the number of processes based on CPU core count
    num_workers = 10

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Using list() to ensure all futures are executed and to get the results
        results = list(tqdm(executor.map(compute_row, [row for _, row in df.iterrows()]), total=len(df)))

    # Combine results back into a single DataFrame
    processed_df = pd.DataFrame(results)
    return processed_df

In [11]:
new_df = process_dataframe_in_parallel(momenta)

new_df = new_df.explode(
    momentum_keys
    + ["is_signal", "production_process", "particle_name", "MCTrack.fPdgCode"] + ["pt", "P", "theta", "phi", "eta"],
    ignore_index=True,
)

new_df.to_parquet(
    "/home/nikin105/mlProject/data/simulations/XiAntiXi/processed/particle_data.parquet",
    index=False,
)

KeyboardInterrupt: 

In [ ]:
new_df.head()

In [ ]:
track_ids = sim_tree["STTPoint.fTrackID"].array(library="pd")
unique_track_ids = []
for entry in track_ids:
    unique_track_ids.append(np.unique(entry))

momentum_keys = [
    "MCTrack.fPx",
    "MCTrack.fPy",
    "MCTrack.fPz",
]

pid_keys = [
    "MCTrack.fMotherID",
    "MCTrack.fSecondMotherID",
    "MCTrack.fProcess",
    "MCTrack.fPdgCode",
]
stt_keys = ["STTPoint.fTrackID"]

momenta = sim_tree.arrays(
    expressions=momentum_keys + pid_keys + stt_keys, library="pd", entry_stop=100
)


for key in momentum_keys:
    momenta[key] = momenta.apply(
        lambda row: [row[key][idx] for idx in unique_track_ids[row.name]],
        axis=1,
    )

momenta["mother_ids"] = momenta.apply(
    lambda row: get_all_mother_ids(
        mother_ids=row["MCTrack.fMotherID"],
        second_mother_ids=row["MCTrack.fSecondMotherID"],
    ),
    axis=1,
)

momenta = momenta.drop(columns=["MCTrack.fMotherID", "MCTrack.fSecondMotherID"])

momenta["process_ids"] = momenta.apply(
    lambda row: [
        get_process_ids(
            process_ids=row["MCTrack.fProcess"],
            mother_ids=row["mother_ids"],
            pdg_ids=row["MCTrack.fPdgCode"],
            particle_id=particle_id,
        )
        for particle_id in unique_track_ids[row.name]
    ],
    axis=1,
)

momenta = momenta.drop(columns=["MCTrack.fProcess", "MCTrack.fPdgCode", "mother_ids"])

momenta["is_signal"] = momenta.apply(
    lambda row: [
        is_signal_particle(
            process_mc_ids=row["process_ids"][particle_id][0],
            process_ids=row["process_ids"][particle_id][1],
            signal_mc_ids=signal_processes,
            signal_process_ids=signal_process_ids,
        )
        for particle_id in range(len(row["process_ids"]))
    ],
    axis=1,
)

momenta["production_process"] = momenta.apply(
    lambda row: [
        get_process_name(process_id=row["process_ids"][particle_id][1][-1])
        for particle_id in range(len(row["process_ids"]))
    ],
    axis=1,
)

momenta["pdg_code"] = momenta.apply(
    lambda row: [
        row["process_ids"][particle_id][0][-1]
        for particle_id in range(len(row["process_ids"]))
    ],
    axis=1,
)

momenta["particle_name"] = momenta.apply(
    lambda row: [
        r"$" + get_particle_tex_name(row["process_ids"][particle_id][0][-1]) + r"$"
        for particle_id in range(len(row["process_ids"]))
    ],
    axis=1,
)

momenta = momenta.drop(columns=["process_ids"])

momenta = momenta.explode(
    momentum_keys + ["is_signal", "production_process", "particle_name", "pdg_code"],
    ignore_index=True,
)

momenta["pt"] = momenta.apply(
    lambda row: np.sqrt(row["MCTrack.fPx"] ** 2 + row["MCTrack.fPy"] ** 2), axis=1
)
momenta["P"] = momenta.apply(
    lambda row: np.sqrt(
        row["MCTrack.fPx"] ** 2 + row["MCTrack.fPy"] ** 2 + row["MCTrack.fPz"] ** 2
    ),
    axis=1,
)
momenta["theta"] = momenta.apply(
    lambda row: np.arctan2(row["pt"], row["MCTrack.fPz"]), axis=1
)
momenta["phi"] = momenta.apply(
    lambda row: np.arctan2(row["MCTrack.fPy"], row["MCTrack.fPx"]), axis=1
)
momenta["eta"] = momenta.apply(lambda row: -np.log(np.tan(row["theta"] / 2)), axis=1)

momenta.head()

In [ ]:
momenta.head()

In [10]:
momenta.to_parquet(
    "/home/nikin105/mlProject/data/simulations/XiAntiXi/processed/particle_data.parquet",
    index=False,
)

In [ ]:
momenta = pd.read_parquet(
    "/home/nikin105/mlProject/data/simulations/XiAntiXi/processed/particle_data.parquet"
)
momenta.head()